In [1]:
import rosbag_pandas
import datetime
import pandas as pd
import numpy as np

In [2]:
bag_file = "raw_ron_data19__2015-09-14-21-01-48.bag"

In [3]:
df = rosbag_pandas.bag_to_dataframe(bag_file)
ebt = df['object_tracking_2d_ros_detections__detections']

ebt_pose__pose_pose_orientation_w = pd.Series()
ebt_pose__pose_pose_orientation_x = pd.Series()
ebt_pose__pose_pose_orientation_y = pd.Series()
ebt_pose__pose_pose_orientation_z = pd.Series()
ebt_pose__pose_pose_position_x = pd.Series()
ebt_pose__pose_pose_position_y = pd.Series()
ebt_pose__pose_pose_position_z = pd.Series()

for i in ebt.index:
    ebt_i = ebt.at_time(i)[0]
    if(ebt_i is not None):
        ebt_i = ebt_i[0]
        ebt_pose__pose_pose_orientation_w.set_value(i, ebt_i.pose.pose.orientation.w)
        ebt_pose__pose_pose_orientation_x.set_value(i, ebt_i.pose.pose.orientation.x)
        ebt_pose__pose_pose_orientation_y.set_value(i, ebt_i.pose.pose.orientation.y)
        ebt_pose__pose_pose_orientation_z.set_value(i, ebt_i.pose.pose.orientation.z)
        ebt_pose__pose_pose_position_x.set_value(i, ebt_i.pose.pose.position.x)
        ebt_pose__pose_pose_position_y.set_value(i, ebt_i.pose.pose.position.y)
        ebt_pose__pose_pose_position_z.set_value(i, ebt_i.pose.pose.position.z)

df['ebt_pose__pose_pose_orientation_w'] = ebt_pose__pose_pose_orientation_w
df['ebt_pose__pose_pose_orientation_x'] = ebt_pose__pose_pose_orientation_x
df['ebt_pose__pose_pose_orientation_y'] = ebt_pose__pose_pose_orientation_y
df['ebt_pose__pose_pose_orientation_z'] = ebt_pose__pose_pose_orientation_z
df['ebt_pose__pose_pose_position_x'] = ebt_pose__pose_pose_position_x
df['ebt_pose__pose_pose_position_y'] = ebt_pose__pose_pose_position_y
df['ebt_pose__pose_pose_position_z'] = ebt_pose__pose_pose_position_z

df = df.drop('object_tracking_2d_ros_detections__detections', 1)

for c in df.columns:
    print c

ground_col = ['Robot_1_pose__pose_position_x',
              'Robot_1_pose__pose_position_y',
              'Robot_1_pose__pose_position_z',
              'Robot_1_pose__pose_orientation_x',
              'Robot_1_pose__pose_orientation_y',
              'Robot_1_pose__pose_orientation_z',
              'Robot_1_pose__pose_orientation_w']
ground_list = [df[ground_col[0]], 
               df[ground_col[1]], 
               df[ground_col[2]], 
               df[ground_col[3]], 
               df[ground_col[4]], 
               df[ground_col[5]], 
               df[ground_col[6]]]
ground = pd.concat(ground_list, axis=1, keys=ground_col)
ground = ground.dropna(how='all')

ground.to_csv('ground.data', sep=" ", header=False, date_format="%s.%f")

svo_col = ['svo_pose__pose_pose_position_x',
           'svo_pose__pose_pose_position_y',
           'svo_pose__pose_pose_position_z',
           'svo_pose__pose_pose_orientation_x',
           'svo_pose__pose_pose_orientation_y',
           'svo_pose__pose_pose_orientation_z',
           'svo_pose__pose_pose_orientation_w']
svo_list = [df[svo_col[0]], 
            df[svo_col[1]], 
            df[svo_col[2]], 
            df[svo_col[3]], 
            df[svo_col[4]], 
            df[svo_col[5]], 
            df[svo_col[6]]]
svo = pd.concat(svo_list, axis=1, keys=svo_col)
svo = svo.dropna(how='all')

svo.to_csv('svo.data', sep=" ", header=False, date_format="%s.%f")

ebt_col = ['ebt_pose__pose_pose_position_x',
           'ebt_pose__pose_pose_position_y',
           'ebt_pose__pose_pose_position_z',
           'ebt_pose__pose_pose_orientation_x',
           'ebt_pose__pose_pose_orientation_y',
           'ebt_pose__pose_pose_orientation_z',
           'ebt_pose__pose_pose_orientation_w']
ebt_list = [df[ebt_col[0]], 
            df[ebt_col[1]], 
            df[ebt_col[2]], 
            df[ebt_col[3]], 
            df[ebt_col[4]], 
            df[ebt_col[5]], 
            df[ebt_col[6]]]
ebt = pd.concat(ebt_list, axis=1, keys=ebt_col)
ebt = ebt.dropna(how='all')

ebt.to_csv('ebt.data', sep=" ", header=False, date_format="%s.%f")

Robot_1_pose__pose_orientation_w
Robot_1_pose__pose_orientation_x
Robot_1_pose__pose_orientation_y
Robot_1_pose__pose_orientation_z
Robot_1_pose__pose_position_x
Robot_1_pose__pose_position_y
Robot_1_pose__pose_position_z
svo_pose__pose_covariance
svo_pose__pose_pose_orientation_w
svo_pose__pose_pose_orientation_x
svo_pose__pose_pose_orientation_y
svo_pose__pose_pose_orientation_z
svo_pose__pose_pose_position_x
svo_pose__pose_pose_position_y
svo_pose__pose_pose_position_z
ebt_pose__pose_pose_orientation_w
ebt_pose__pose_pose_orientation_x
ebt_pose__pose_pose_orientation_y
ebt_pose__pose_pose_orientation_z
ebt_pose__pose_pose_position_x
ebt_pose__pose_pose_position_y
ebt_pose__pose_pose_position_z


In [4]:
%%bash
python rgbd_benchmark_tools/evaluate_ate.py ground.data ebt.data --plot ebt_ate.pdf --save ebt_ate_error.data --label "Tracker"

0.095077


In [5]:
%%bash
python rgbd_benchmark_tools/evaluate_rpe.py ground.data ebt.data --plot ebt_rpe.pdf --save ebt_rpe_error.data --fixed_delta

0.0906925796819


In [6]:
%%bash
python rgbd_benchmark_tools/evaluate_ate.py ground.data svo.data --plot svo_ate.pdf --save svo_ate_error.data --label "SVO"

0.100137


In [7]:
%%bash
python rgbd_benchmark_tools/evaluate_rpe.py ground.data svo.data --plot svo_rpe.pdf --save svo_rpe_error.data --fixed_delta

0.208716320773
